In [1]:
import pandas as pd

te = pd.read_csv('text_emotion.csv')

text = te.iloc[:,3].to_string(header=False, index=False)

lines = te.iloc[:, 3:4].values.flatten().tolist()

text

lines

['@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[',
 'Layin n bed with a headache  ughhhh...waitin on your call...',
 'Funeral ceremony...gloomy friday...',
 'wants to hang out with friends SOON!',
 '@dannycastillo We want to trade with someone who has Houston tickets, but no one will.',
 "Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends",
 "I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp; he wants me 2! scandalous!",
 'Hmmm. http://www.djhero.com/ is down',
 '@charviray Charlene my love. I miss you',
 "@kelcouch I'm sorry  at least it's Friday?",
 'cant fall asleep',
 'Choked on her retainers',
 'Ugh! I have to beat this stupid song to get to the next  rude!',
 '@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol',
 'Got the news',
 'The storm is here and the elec

In [2]:
from keras.preprocessing.text import text_to_word_sequence

words = text_to_word_sequence(text)

In [3]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words)
tokens = tokenizer.word_index

In [4]:
vocabulary_size = len(tokens) + 1
print(vocabulary_size)

39283


In [5]:
sequences = tokenizer.texts_to_sequences(lines)

In [6]:
subsequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        subsequence = sequence[:i+1]
        subsequences.append(subsequence)
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequence_length = max([len(sequence) for sequence in sequences])
sequences = pad_sequences(subsequences, maxlen=sequence_length, padding='pre')

In [7]:
x, y = sequences[:,:-1],sequences[:,-1]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [8]:
print(y)

[   1   54    1 ...  426 1315 1658]


In [17]:
from keras.models import Sequential
model = Sequential()

from keras.layers import Embedding
model.add(Embedding(vocabulary_size, 10, input_length=sequence_length))

from keras.layers import LSTM
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=True))

from keras.layers import Dense
model.add(Dense(units=100, activation='relu'))

from keras.layers import Dropout
model.add(Dropout(0.5))


from keras.layers import Dense
model.add(Dense(units=5, activation='softmax'))

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, batch_size=256, epochs = 10)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 35, 10)            392830    
                                                                 
 lstm_10 (LSTM)              (None, 35, 128)           71168     
                                                                 
 lstm_11 (LSTM)              (None, 35, 64)            49408     
                                                                 
 dense_6 (Dense)             (None, 35, 100)           6500      
                                                                 
 dropout_3 (Dropout)         (None, 35, 100)           0         
                                                                 
 dense_7 (Dense)             (None, 35, 5)             505       
                                                                 
Total params: 520,411
Trainable params: 520,411
Non-tr

ValueError: in user code:

    File "c:\Anaconda\envs\images\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Anaconda\envs\images\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Anaconda\envs\images\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Anaconda\envs\images\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Anaconda\envs\images\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Anaconda\envs\images\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of laye